# House_Price_Prediction — Global Forecasting LightGBM

In [147]:
!pip install darts

In [148]:
import matplotlib.pyplot as plt
#plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['font.family'] ='NanumGothic'
plt.rcParams['axes.unicode_minus'] =False

In [149]:
import os
                        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from darts import TimeSeries
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, InvertibleMapper, StaticCovariatesTransformer
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller
from darts.metrics import rmsle
from darts.models import LinearRegressionModel, LightGBMModel, XGBModel, CatBoostModel
from darts.models.filtering.moving_average_filter import MovingAverageFilter
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tqdm.notebook import tqdm_notebook

plt.style.use("ggplot")
plt.rcParams["font.size"] = 15
COLORS = list(sns.color_palette())

# 1. Data Preprocessing

In [150]:
# helper function to print messages
def cprint(title, *args):
    print(
        "="*len(title), title, "="*len(title),
        *args,
        sep="\n",
    )

### Loading the datasets.

In [166]:
PATH = r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data"

train = pd.read_csv(os.path.join(PATH, "train.csv")) # (1118822, 52)
test = pd.read_csv(os.path.join(PATH, "test.csv")) # (9272, 51)
train_test = pd.read_csv(os.path.join(PATH, "train_test.csv")) # (1128094, 53)
subway_feature = pd.read_csv(os.path.join(PATH, "subway_feature.csv"))
sample_submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
bus_feature = pd.read_csv(os.path.join(PATH, "bus_feature.csv"))
seoul_apart = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\seoul_apart.csv", encoding='utf-8') # (2792, 47)
# 출처: https://data.seoul.go.kr/dataList/OA-15818/S/1/datasetView.do 서울시 공동주택 아파트 정보
doroaddress = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\doroaddress.csv", encoding='utf-8') # (8323, 5)
bunjiaddress = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\bunjiaddress.csv", encoding='utf-8') # (8954, 1) 
# 출처: https://www.juso.go.kr/CommonPageLink.do?link=/support/AddressTransform 100만건 이하 주소전환

C:\Users\user\AppData\Local\Temp\ipykernel_22368\3119428652.py:3: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(os.path.join(PATH, "train.csv")) # (1118822, 52)
C:\Users\user\AppData\Local\Temp\ipykernel_22368\3119428652.py:5: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_test = pd.read_csv(os.path.join(PATH, "train_test.csv")) # (1128094, 53)


## read_csv 만들기 위한 데이터전처리 과정 --> 완료

### bunjiaddress

In [ ]:
import pandas as pd

# Assuming 'train_test' is your DataFrame
# Extract unique values from '번지주소' column and convert them to a DataFrame
unique_addresses = pd.DataFrame(train_test['번지주소'].unique(), columns=['Unique Addresses'])

# Save to a CSV file with cp949 encoding, no index, and no header
unique_addresses.to_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\번지주소.txt", encoding='cp949', index=False, header=False)


In [ ]:
doroaddress = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\도로명주소.txt", encoding='cp949') # 
bunjiaddress = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\번지주소.txt", encoding='cp949') # 

doroaddress.to_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\도로명주소.txt", encoding='utf-8', index=False)
bunjiaddress.to_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\번지주소.txt", encoding='utf-8', index=False)


In [152]:
bunjiaddress.rename(columns={0:'번지주소'}, inplace=True)

In [153]:
bunjiaddress = bunjiaddress.drop(bunjiaddress.columns[0], axis=1)

In [ ]:
bunjiaddress = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\번지주소.txt", encoding='cp949', header=None) #  


In [ ]:
bunjiaddress.to_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\bunjiaddress.csv", index=False)

In [156]:
bunjiaddress

,번지주소
0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동 652
2,서울특별시 강남구 개포동 12-2
3,서울특별시 강남구 개포동 141
4,서울특별시 강남구 개포동 187
...,...
8949,서울특별시 동작구 흑석동 253-89
8950,서울특별시 마포구 신수동 400-1
8951,서울특별시 성동구 송정동 67-5
8952,서울특별시 중구 인현동2가 240


### doroaddress

In [ ]:
column_names = ['결과코드', '우편번호', '지번주소', '도로명주소', '건물관리번호']
doroaddress = pd.read_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\도로명주소.txt", delimiter=';', names=column_names, on_bad_lines='warn')

In [ ]:
doroaddress = doroaddress.drop(doroaddress.index[0])

In [ ]:
doroaddress = doroaddress.drop(doroaddress.columns[0], axis=1)

In [ ]:
doroaddress.to_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\doroaddress.csv", index=False)

In [157]:
doroaddress

,결과코드,우편번호,지번주소,도로명주소,건물관리번호
0,전환성공,6316.0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3 (개포동),1168010300106580001019759
1,전환성공,6319.0,서울특별시 강남구 개포동 652,서울특별시 강남구 개포로 307 (개포동),1168010300106520000019285
2,전환성공,6334.0,서울특별시 강남구 개포동 12-2,서울특별시 강남구 개포로109길 69 (개포동),1168010300100120002000001
3,자료없음,0.0,서울특별시 강남구 개포동 141,NaN,NaN
4,전환성공,6330.0,서울특별시 강남구 개포동 185,서울특별시 강남구 개포로 516 (개포동),1168010300101850000019804
...,...,...,...,...,...
8318,전환성공,6987.0,서울특별시 동작구 흑석동 253-89,서울특별시 동작구 서달로 90 (흑석동),1159010500102530089016034
8319,전환성공,4088.0,서울특별시 마포구 신수동 400-1,서울특별시 마포구 토정로17길 6 (신수동),1144011100104000001024567
8320,전환성공,4802.0,서울특별시 성동구 송정동 67-5,서울특별시 성동구 송정길 32 (송정동),1120011800100670005003280
8321,전환성공,4548.0,서울특별시 중구 인현동2가 240,서울특별시 중구 마른내로 79 (인현동2가),1114013400101510001014682


In [ ]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100) 

### train_test

In [ ]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
train_test = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [ ]:
train_test['번지주소'] = train_test['시군구'] + ' ' + train_test['번지']

In [165]:
train_test = train_test.drop(train_test.columns[0], axis=1)

In [161]:
train_test.to_csv(r"C:\Users\user\upstage3-python\12.upstage_project_House_Price_Prediction\asset\data\train_test.csv", index=False)

In [168]:
train_test.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,번지주소
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0,서울특별시 강남구 개포동 658-1


### seoul_apart

In [169]:
seoul_apart.head(1)

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),주소(도로상세주소),k-전화번호,k-팩스번호,단지소개기존clob,단지첨부파일,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,1,A15679103,우리유앤미,아파트,서울특별시 동작구 서달로 83,서울,동작구,흑석동,우리유앤미아파트,서달로,83,28127541,28127542,NaN,NaN,분양,위탁관리,혼합식,개별난방,2.0,206.0,우리건설,경수재건축조합,0.0,27097.0,15827.0,20098.0,89.0,93.0,24.0,NaN,NaN,NaN,0.005012,90800610361,위탁,단일계약,위탁,1773.56,223.0,의무,0.041458,Y,N,126.9596386,37.500668,0.032627


## 결측치

In [7]:
train.isna().sum()

시군구                             0
번지                            225
본번                             75
부번                             75
아파트명                         2126
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1112839
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         870691
k-전화번호                     870274
k-팩스번호                     872742
단지소개기존clob                1050240
k-세대타입(분양형태)               869563
k-관리방식                     869563
k-복도유형                     869890
k-난방방식                     869563
k-전체동수                     870630
k-전체세대수                    869563
k-건설사(시공사)                 871058
k-시행사                      871254
k-사용검사일-사용승인일              869696
k-연면적                      869563
k-주거전용면적      

In [9]:
train.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')

In [10]:
test.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일'],
      dtype='object')

In [13]:
test.head(5)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,언주로 3,NaN,,직거래,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,개포로 311,NaN,,중개거래,서울 강남구,아파트,025763999,025763998,NaN,분양,위탁관리,계단식,지역난방,2.0,232.0,포스코,개포우성9차모델링조합,2021-12-01 00:00:00.0,44951.0,24878.0,31174.0,0.0,0.0,232.0,NaN,thesharp-trieh.hthomeservice.com/#/feeManage?_...,2022-01-05 18:02:23.0,2023-09-23 17:23:08.0,NaN,위탁,종합계약,위탁,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,개포로 307,NaN,,중개거래,서울 강남구,아파트,025749116,025749094,2888.0,분양,자치관리,혼합식,지역난방,5.0,405.0,우성건설,우성건설,1984-12-31 00:00:00.0,63304.0,53437.0,61064.0,0.0,0.0,285.0,NaN,NaN,NaN,2023-09-23 17:15:43.0,21380015910,직영,단일계약,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,개포로 307,NaN,,중개거래,서울 강남구,아파트,025749116,025749094,2888.0,분양,자치관리,혼합식,지역난방,5.0,405.0,우성건설,우성건설,1984-12-31 00:00:00.0,63304.0,53437.0,61064.0,0.0,0.0,285.0,NaN,NaN,NaN,2023-09-23 17:15:43.0,21380015910,직영,단일계약,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,개포로 307,NaN,,중개거래,서울 강남구,아파트,025749116,025749094,2888.0,분양,자치관리,혼합식,지역난방,5.0,405.0,우성건설,우성건설,1984-12-31 00:00:00.0,63304.0,53437.0,61064.0,0.0,0.0,285.0,NaN,NaN,NaN,2023-09-23 17:15:43.0,21380015910,직영,단일계약,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
